In [5]:
import pickle
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import Conv2D
from keras.layers import Dense, Activation, Flatten, Input, Dropout
from tensorflow.keras.optimizers import SGD
from keras.models import load_model
import cv2 as cv
from keras.preprocessing.image import load_img, img_to_array
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
def load_data():
    file = open('/content/drive/MyDrive/dataCNN/foods.data', 'rb')

    # dump information to that file
    (pixels, labels) = pickle.load(file)

    # close the file
    file.close()

    print(pixels.shape)
    print(labels.shape)


    return pixels, labels

In [79]:
label = ['Banh cuon','Banh mi','Banh xeo','Bun bo hue','Bun dau mam tom','Chao long','Com tam','Goi cuon','Mi quang','Pho']

In [40]:
X,y = load_data()

(5500, 128, 128, 3)
(5500, 10)


In [56]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=100)

In [57]:
model = Sequential()

In [58]:
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(128, 128, 3)))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [59]:
opt = SGD(lr=0.001, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.1,
    rescale=1./255,
	width_shift_range=0.1,
    height_shift_range=0.1,
	horizontal_flip=True,
    brightness_range=[0.2,1.5], fill_mode="nearest")

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [60]:
history = model.fit(aug.flow(X_train, y_train, batch_size=128),epochs=50, validation_data=aug.flow(X_test,y_test,batch_size=128), verbose=1)

Epoch 1/50
35/35 [==============================] - 26s 737ms/step - loss: 2.2943 - accuracy: 0.1384 - val_loss: 2.2397 - val_accuracy: 0.2127
Epoch 2/50
35/35 [==============================] - 24s 689ms/step - loss: 2.1615 - accuracy: 0.2291 - val_loss: 2.1907 - val_accuracy: 0.2182
Epoch 3/50
35/35 [==============================] - 25s 723ms/step - loss: 2.0824 - accuracy: 0.2548 - val_loss: 2.1220 - val_accuracy: 0.2691
Epoch 4/50
35/35 [==============================] - 24s 689ms/step - loss: 2.0257 - accuracy: 0.2780 - val_loss: 2.0271 - val_accuracy: 0.2955
Epoch 5/50
35/35 [==============================] - 24s 689ms/step - loss: 2.0233 - accuracy: 0.2884 - val_loss: 2.0645 - val_accuracy: 0.2882
Epoch 6/50
35/35 [==============================] - 24s 692ms/step - loss: 1.9753 - accuracy: 0.3080 - val_loss: 1.9710 - val_accuracy: 0.3145
Epoch 7/50
35/35 [==============================] - 24s 682ms/step - loss: 1.9524 - accuracy: 0.3141 - val_loss: 1.9385 - val_accuracy: 0.3391

In [61]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss: %.4f'% score[0])
print('Test accuracy %.4f'% score[1])

Test loss: 174.1596
Test accuracy 0.5227


In [62]:
model.save('Final.h5')

In [82]:
model_5 = load_model('Final.h5')

In [83]:
img = cv.imread('36.jpg')

In [84]:
img = cv.resize(img, dsize=(128, 128))
img = img.astype('float')*1./255
# Convert to tensor
img = np.expand_dims(img, axis=0)

In [85]:
predict = model_5.predict(img)

In [86]:
label[np.argmax(model.predict(img), axis=1)[0]]

'Mi quang'